# Magic Hands
---

In this project, we are going to work on our first Deep Learning project where we are going to train a Deep Neural Network to recognize your hand patterns. This can be used to create a simple game or control system where based on the detected pattern, you can execute a certain script that will perform some gesture-specific action.

We are using the Tensorflow framework for this project.

## Importing Project Dependencies
---

As the first step of this project, we are going to import all the necessary project dependencies.

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf